In [248]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 400
pd.options.display.max_rows = 400

In [249]:
bd_train=pd.read_csv(r"C:\Users\anvay\Desktop\Data\Consumer_Complaints_train.csv")
bd_test=pd.read_csv(r"C:\Users\anvay\Desktop\Data\Consumer_Complaints_test_share.csv")

In [250]:
bd_test['Consumer disputed?']=np.nan

In [251]:
bd_train['data'] = 'train'
bd_test['data'] = 'test'

In [252]:
bd_test=bd_test[bd_train.columns]

In [253]:
bd_all=pd.concat([bd_train,bd_test],axis=0)

In [254]:
bd_all.head(5)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train


In [255]:
bd_all['Date received']=pd.to_datetime(bd_all['Date received'],infer_datetime_format=True)
bd_all['Date sent to company']=pd.to_datetime(bd_all['Date sent to company'],infer_datetime_format=True)

In [256]:
bd_all['day_diff']=pd.to_numeric(bd_all['Date sent to company']-bd_all['Date received'])

In [257]:
bd_all.drop( ['Date received','Date sent to company'],1,inplace=True)

In [258]:
bd_all.nunique()

Product                             12
Sub-product                         47
Issue                               95
Sub-issue                           68
Consumer complaint narrative     92118
Company public response             10
Company                           3437
State                               62
ZIP code                         27342
Tags                                 3
Consumer consent provided?           4
Submitted via                        6
Company response to consumer         7
Timely response?                     2
Consumer disputed?                   2
Complaint ID                    598027
data                                 2
day_diff                           420
dtype: int64

In [259]:
bd_all.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
day_diff                             0
dtype: int64

In [260]:
bd_all['day_diff']=bd_all['day_diff']//1000000000

In [261]:
bd_all.head(5)

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data,day_diff
0,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,Closed with explanation,Yes,No,856103,train,86400
1,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,Closed,Yes,No,1034666,train,518400
2,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,Closed with non-monetary relief,Yes,No,756363,train,1814400
3,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,Closed with monetary relief,Yes,No,1474177,train,0
4,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,Closed with explanation,Yes,No,1132572,train,691200


In [262]:
bd_all['Issue']=bd_all['Issue'].str.lower()

In [263]:
bd_all['Issue']=bd_all['Issue'].str.replace(","," ").replace('-','_').replace('?','').replace('/','').replace(' / ','').replace("''","").replace(" ",'_')

In [264]:
#temp=bd_all['Issue'].str.contains("loan")

In [265]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [266]:
tfidf = TfidfVectorizer(analyzer='word', stop_words = 'english')
bd_all['Issue_tfidf'] = list(tfidf.fit_transform(bd_all['Issue']).toarray())

In [267]:
bd_all['Issue_max_tfidf']=list(map(max,bd_all['Issue_tfidf']))

In [268]:
bd_all.drop(['Issue','Issue_tfidf'],axis=1,inplace=True)

In [269]:
#bd_all['Issue_tfidf'] = list(score.toarray())

In [270]:
bd_all['Issue_max_tfidf'].head(5)

0    0.812661
1    0.485491
2    0.525690
3    0.812661
4    0.730544
Name: Issue_max_tfidf, dtype: float64

In [271]:
#df = pd.DataFrame(score.toarray(), columns=tfidf.get_feature_names())
#tokens_above_threshold =bd_all['Issue_tfidf'].max().any()

In [272]:
bd_all.dtypes

Product                          object
Sub-product                      object
Sub-issue                        object
Consumer complaint narrative     object
Company public response          object
Company                          object
State                            object
ZIP code                         object
Tags                             object
Consumer consent provided?       object
Submitted via                    object
Company response to consumer     object
Timely response?                 object
Consumer disputed?               object
Complaint ID                      int64
data                             object
day_diff                          int64
Issue_max_tfidf                 float64
dtype: object

In [273]:
bd_all['Submitted via'].value_counts()

Web            392364
Referral       114231
Phone           42958
Postal mail     39348
Fax              8788
Email             337
Name: Submitted via, dtype: int64

In [274]:
bd_all.nunique()

Product                             12
Sub-product                         47
Sub-issue                           68
Consumer complaint narrative     92118
Company public response             10
Company                           3437
State                               62
ZIP code                         27342
Tags                                 3
Consumer consent provided?           4
Submitted via                        6
Company response to consumer         7
Timely response?                     2
Consumer disputed?                   2
Complaint ID                    598027
data                                 2
day_diff                           420
Issue_max_tfidf                     83
dtype: int64

In [275]:
bd_all.isnull().sum()

Product                              0
Sub-product                     173225
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
day_diff                             0
Issue_max_tfidf                      0
dtype: int64

In [276]:
bd_all.drop(['Tags','Consumer consent provided?'],axis=1,inplace=True)

In [277]:
bd_all.drop(['Consumer complaint narrative','Company public response'],axis=1,inplace=True)

In [278]:
del bd_all['Sub-issue']

In [279]:
del bd_all['Complaint ID']

In [280]:
bd_all.drop(['ZIP code'],axis=1,inplace=True)

In [281]:
del bd_all['Sub-product']

In [282]:
bd_all.dtypes

Product                          object
Company                          object
State                            object
Submitted via                    object
Company response to consumer     object
Timely response?                 object
Consumer disputed?               object
data                             object
day_diff                          int64
Issue_max_tfidf                 float64
dtype: object

In [283]:
bd_all.head(5)

,Product,Company,State,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,data,day_diff,Issue_max_tfidf
0,Credit card,Wells Fargo & Company,MI,Web,Closed with explanation,Yes,No,train,86400,0.812661
1,Bank account or service,Santander Bank US,PA,Referral,Closed,Yes,No,train,518400,0.485491
2,Credit reporting,Equifax,CA,Referral,Closed with non-monetary relief,Yes,No,train,1814400,0.525690
3,Credit card,U.S. Bancorp,GA,Web,Closed with monetary relief,Yes,No,train,0,0.812661
4,Credit card,Bank of America,MA,Web,Closed with explanation,Yes,No,train,691200,0.730544


In [284]:
bd_all['Timely response?'].value_counts()

Yes    587954
No      10073
Name: Timely response?, dtype: int64

In [285]:
bd_all['Timely response?']=bd_all['Timely response?'].map({"Yes":1, "No":0})

In [286]:
bd_all['Timely response?'].value_counts()

1    587954
0     10073
Name: Timely response?, dtype: int64

In [287]:
bd_all['Consumer disputed?']=bd_all['Consumer disputed?'].map({"Yes":1,"No":0})

In [288]:
bd_all['Consumer disputed?'].value_counts()

0.0    376990
1.0    101431
Name: Consumer disputed?, dtype: int64

In [289]:
bd_all['Product'].value_counts()

Mortgage                   195079
Debt collection            108047
Credit reporting           101612
Credit card                 71613
Bank account or service     68094
Consumer Loan               23263
Student loan                18635
Money transfers              4176
Payday loan                  4048
Prepaid card                 2813
Other financial service       635
Virtual currency               12
Name: Product, dtype: int64

In [290]:
bd_all.corr()

,Timely response?,Consumer disputed?,day_diff,Issue_max_tfidf
Timely response?,1.000000,0.003346,-0.010921,0.009824
Consumer disputed?,0.003346,1.000000,-0.025214,0.001532
day_diff,-0.010921,-0.025214,1.000000,-0.022119
Issue_max_tfidf,0.009824,0.001532,-0.022119,1.000000


In [291]:
len(bd_train)

478421

In [292]:
len(bd_all)

598027

In [293]:
bd_all.select_dtypes('object').columns

Index(['Product', 'Company', 'State', 'Submitted via',
       'Company response to consumer', 'data'],
      dtype='object')

In [295]:
for col in ['Product', 'Company', 'State', 'Submitted via','Company response to consumer']:
    temp=pd.get_dummies(bd_all[col],prefix=col)
    bd_all=pd.concat([bd_all,temp],axis=1)
    bd_all.drop([col],axis=1,inplace=True)

In [ ]:
bd_all.dtypes

In [297]:
bd_train=bd_all[bd_all['data']=="train"]

In [231]:
del bd_train['data']

In [232]:
train.head(4)

,Timely response?,Consumer disputed?,day_diff,Issue_max_tfidf,Product_Bank account or service,Product_Consumer Loan,Product_Credit card,Product_Credit reporting,Product_Debt collection,Product_Money transfers,Product_Mortgage,Product_Other financial service,Product_Payday loan,Product_Prepaid card,Product_Student loan,Product_Virtual currency,Submitted via_Email,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,Submitted via_Referral,Submitted via_Web,Company response to consumer_Closed,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response
0,1,0.0,86400,0.812661,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
1,1,0.0,518400,0.485491,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,1,0.0,1814400,0.525690,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3,1,0.0,0,0.812661,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


In [233]:
bd_test=bd_all[bd_all['data']=='test']

In [234]:
bd_test.drop(['Consumer disputed?','data'],axis=1,inplace=True)

C:\Users\anvay\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [235]:
from sklearn.model_selection import train_test_split

train1,train2=train_test_split(train,test_size=0.2,random_state=2)

In [236]:
x_train1=train1.drop('Consumer disputed?',axis=1)
y_train1=train1['Consumer disputed?']

In [174]:
from sklearn.linear_model import LinearRegression

lm= LinearRegression()

lm.fit(x_train1,y_train1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [237]:
from sklearn.linear_model import LogisticRegression

clf=LogisticRegression()

In [238]:
clf.fit(x_train1,y_train1)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [177]:
#bd_test=pd.read_csv(r"C:\Users\anvay\Desktop\Data\Consumer_Complaints_test_share.csv")

In [239]:
x_train2=train2.drop('Consumer disputed?',axis=1)

predicted_ir=lm.predict(x_train2)

In [240]:
x_train=train.drop('Consumer disputed?',axis=1)
y_train=train['Consumer disputed?']
clf.fit(x_train,y_train)

test_pred=clf.predict(bd_test)

In [245]:
test=pd.read_csv(r"C:\Users\anvay\Desktop\Data\Consumer_Complaints_test_share.csv")

In [246]:
test.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Complaint ID'],
      dtype='object')

In [247]:
submission=pd.DataFrame(list(zip(test['Complaint ID'],list(test_pred))),
                       columns=['Complaint ID','Consumer disputed?'])

pd.DataFrame(submission).to_csv("Sample_Submission.csv",index=False)